In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################################
#SETUP

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
num_jobs=60 #1M parcels
# num_jobs=200 #50M parcels
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
job_id=1
[start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

In [ ]:
# Reading Back Data Later
##############

def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 100_000} 
        )
        data_dict = {k: in_data[k][start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def MakeDictionary(**vars):
    return vars
    
def GetData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['W', 'QCQI']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    W, QCQI = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['QV','TH','TH_E','BUOYANCY','HMC']#,'QI','QR']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    QV, TH, TH_E, BUOYANCY, HMC = [data_dict[k] for k in var_names] #, QI, QR

    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['VMF_c','VMF_g']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    VMF_c, VMF_g = [data_dict[k] for k in var_names]


    VARs=MakeDictionary(W=W, QCQI=QCQI, QV=QV, 
                        TH=TH, TH_E=TH_E, 
                        BUOYANCY=BUOYANCY, HMC=HMC,
                        VMF_g=VMF_g,VMF_c=VMF_c)
    return VARs

In [ ]:
def GetSpatialData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['A_g','A_c','Z', 'Y', 'X', 'z']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    A_g,A_c, Z,Y,X,parcel_z = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    return A_g,A_c, Z,Y,X,parcel_z

In [ ]:
def GetSubsetParams(data1):
    #DOMAIN SUBSETTING
    ############################################################
    #FINDING NEW LEFT T-BOUNDARY
    dt=data1['time'][1].item()/1e9 #seconds per timestep
    dhours=(dt/60**2) #hours per timestep
    start_hour=4 #10am
    t_start=int(start_hour/dhours)
    #FINDING NEW RIGHT T-BOUNDARY
    end_hour=11 #5pm
    t_end=int(end_hour/dhours)
    #PRINTING
    # print(f't in {t_start}:{t_end}')
    
    
    #FINDING NEW TOP Z-BOUNDARY
    dzh=data1['zh']
    zh_bottom=0; zh_top=np.where(dzh>=19)[0][0]
    dzf=data1['zf']
    zf_bottom=0; zf_top=np.where(dzf>=20)[0][0]
    #PRINTING
    # print(f'zh_top at {zh_bottom}:{zh_top}')
    
    #FINDING THE NEW LEFT X-BOUNDARY
    ocean_percent=2/8
    left_to_coast=data1['xh'][0]+(data1['xh'][-1]-data1['xh'][0])*ocean_percent
    where_coast_xh=np.where(data1['xh']>=left_to_coast)[0][0]#-25
    where_coast_xf=np.where(data1['xf']>=left_to_coast)[0][0]#-25
    #FINDING THE NEW RIGHT X-BOUNDARY
    right_fraction=80/100
    xf=data1['xf']-data1['xf'][0]
    Nxf=len(xf)
    end_xf=np.where(xf>Nxf*right_fraction)[0][0]
    xh=data1['xh']-data1['xh'][0]
    Nxh=len(xh)
    end_xh=np.where(xh>Nxh*right_fraction)[0][0]
    #PRINTING
    # print(f'x in {0}:{where_coast_xh-1} FOR SEA')
    # print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
    ############################################################
    
    return (
        t_start, t_end,
        zh_bottom, zf_bottom,
        zh_top, zf_top,
        where_coast_xh, where_coast_xf,
        end_xh, end_xf
    )
# [t_start, t_end, zh_bottom, zf_bottom, zh_top, zf_top, where_coast_xh, where_coast_xf, end_xh, end_xf] = GetSubsetParams(data1)

In [ ]:
def DomainSubsetting(A_g,A_c):
    [t_start, t_end, zh_bottom, zf_bottom, zh_top, zf_top, where_coast_xh, where_coast_xf, end_xh, end_xf] = GetSubsetParams(data1)
    ############################################################
    
    #SUBSETTING CODE
    cond1=((X<where_coast_xh)|(X>end_xh))
    cond2=((Z<zh_bottom)|(Z>zh_top))
    A_g[cond1&cond2]=0
    A_c[cond1&cond2]=0
    
    #SUBSETTING TIME FOR JOB ARRAY
    if end_job<=t_start:
        A_g[0:t_start]=0
        A_c[0:t_start]=0
    return A_g,A_c

In [ ]:
##############################################
#MAKE PROFILES FUNCTIONS

In [ ]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'

def LagrangianProfiles_Simultaneous(VARs,Z,data_type):
    # print(f'currently working on type {data_type}')

    #MAKE PROFILE_ARRAYS TO HOLD THE RESULT
    profile_dict = {}
    zhs = data1['zh'].values  # outside the loop for efficiency
    for var in VARs:
        profile_dict[f"profile_{var}"]=np.zeros((len(zhs), 3))
        profile_dict[f"profile_{var}"][:,2]=zhs
    
    #GET THE LOCATIONS OF PARCELS SUCCEEDING THRESHOLD
    if data_type=='general':
        where=np.where(A_g==1)
        zs=Z[:,where[0]]
    elif data_type=='cloudy':
        where=np.where(A_c==1)
        zs=Z[:,where[0]]

    #For Each Variable, Link profile_array to profile_{var} and Add Profile Elements to Storage Array
    for var in VARs:
        var_data = VARs[var][:,where[0]]
        np.add.at(profile_dict[f"profile_{var}"][:, 0], zs, var_data)
        np.add.at(profile_dict[f"profile_{var}"][:, 1], zs, 1)
    return profile_dict

In [ ]:
def SaveProfile(Dictionary,data_type):
    # print("Saving all profiles...")

    #OUTPUTTING RESULTS    
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    if data_type=='general':
        output_file=dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}'
    elif data_type=='cloudy':
        output_file=dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}'
    output_file+=f'_{job_id}.h5'

    # Write the entire dictionary to HDF5
    with h5py.File(output_file, "w") as h5f:
        for key, profile_data in Dictionary.items():
            h5f.create_dataset(key, data=profile_data)
            
    # print("Done saving.\n")

In [ ]:
##############################################
#RUNNING

In [ ]:
num_slurm_jobs=30
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    data=data1.isel(time=slice(start_job,end_job))
    parcel=parcel1.isel(xh=slice(start_job,end_job))

    #GETTING DATA AND PUTTING IN A DICTIONARY
    [A_g,A_c, Z,Y,X,parcel_z] = GetSpatialData(start_job,end_job) 
    [A_g,A_c]=DomainSubsetting(A_g,A_c)
    VARs=GetData(start_job,end_job)

    #CALCULATING AND SAVING
    for data_type in ['general','cloudy']:
        Dictionary=LagrangianProfiles_Simultaneous(VARs, Z, data_type) #VERSION2: EACH VARIABLE SIMULTANEOUSLY (RECOMMENDED)
        SaveProfile(Dictionary,data_type)
        del Dictionary
    #check_memory(globals())
    del VARs

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
recombine=True

In [ ]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'
def GetInputFileName(data_type,job_id):
    #CALLING IN DATA
    if data_type == "general":
        input_file = dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
    elif data_type == "cloudy":
        input_file = dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
    return input_file
def GetOutputFileName(data_type):
    #CALLING IN DATA
    if data_type == "general":
        output_file = dir2+f'job_out/general_lagrangian_profiles_{res}_{t_res}_{Np_str}.h5' 
    elif data_type == "cloudy":
        output_file = dir2+f'job_out/cloudy_lagrangian_profiles_{res}_{t_res}_{Np_str}.h5'
    return output_file
def Recombine(num_jobs):
    data_types=["general","cloudy"]
    for data_type in data_types:
        
        #MAKING OUTPUT FILE PATH
        output_file=GetOutputFileName(data_type)
        
        #MAKING PROFILES DICTIONARY
        profiles = {}  # Store profiles for all variables
        job_id=1; input_file=GetInputFileName(data_type,job_id)
        with h5py.File(input_file, 'r') as f:
            for var in f:
                profiles[var]=f[f"{var}"][:]

        #RECOMBINING
        for job_id in np.arange(2,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
            #CALLING IN DATA and COMPILING PROFILES
            input_file=GetInputFileName(data_type,job_id)
            with h5py.File(input_file, 'r') as f:
                for var in f:  

                    profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    Recombine(num_jobs=num_jobs)

In [ ]:
# #TESTING
# def averaged_profiles(profile):
#     out_var = profile[(profile[:, 1] != 0)]  # gets rid of rows that have no data
#     out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T  # divides the data column by the counter column
#     return out_var
# job_id=20; input_file=GetInputFileName(data_type,job_id)
# with h5py.File(input_file, 'r') as f:
#     profile=f['profile_W'][:]
# out=averaged_profiles(profile)
# plt.plot(out[:,0],out[:,1])